<a href="https://colab.research.google.com/github/kate-simonova/cancer-subtyping/blob/main/notebooks/03_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Data Loading and installation of packages

In [ ]:
!pip install umap-learn
!pip install imblearn

In [ ]:
# importing necessary packages
from google.colab import drive
drive.mount('/content/drive') 
path = "/content/drive/MyDrive/MY_DATA/merged"

import re

import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import ADASYN
from collections import Counter
from sklearn.preprocessing import LabelEncoder

from sklearn.cluster import KMeans
from sklearn.decomposition import KernelPCA, PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.model_selection import KFold
from keras.layers import Layer, Activation
from keras.callbacks import Callback

import keras
keras.backend.clear_session()

import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU, Lambda
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import BatchNormalization

from sklearn.preprocessing import MinMaxScaler

MessageError: ignored

## Loading data 

Data were previosuly generated in [data splitting step](https://colab.research.google.com/drive/1FoqO_K2jZzqqD49h6zzHFwUWCHT_Ddg4).

In [ ]:
test_df = pd.read_csv(f"{path}/split_circle/X_test_A_merged_CRC_BRCA.csv.tar.gz", compression = "gzip", index_col=0).T
test_df.shape

KeyboardInterrupt: ignored

In [ ]:
y_test = pd.read_csv(f"{path}/split_circle/y_test_A_merged_CRC_BRCA.csv.tar.gz", compression = "gzip", index_col=0)
#y_test = y_test.reset_index(drop=True)
#y_test.index = test_df.index
y_test.shape

In [ ]:
train_df = pd.read_csv(f"{path}/split_circle/X_train_A_merged_CRC_BRCA.csv.tar.gz", compression="gzip", index_col=0).T
train_df.shape

In [ ]:
y_train = pd.read_csv(f"{path}/split_circle/y_train_A_merged_CRC_BRCA.csv.tar.gz", compression = "gzip", index_col=0)
#y_train = y_train.reset_index(drop=True)
#y_train.index = train_df.index
y_train.shape

# Dimensionality reduction with PCA

In [ ]:
pca = PCA(0.95).fit(train_df)
x_train = pca.transform(train_df)
x_test = pca.transform(test_df)
print(x_train.shape)
print(x_test.shape)

In [ ]:
x_test = pd.DataFrame(x_test, index=test_df.index)
x_train = pd.DataFrame(x_train, index=train_df.index)
x_test.shape

In [ ]:
x_test.columns = ["PC"+ str(i+1) for i in range(4557)]

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(x_train, y_train["Label"], test_size=0.2, random_state=42, stratify=y_train["Label"])
X_tr.shape

# Oversampling and synthetic data generation

In [ ]:
mapp = LabelEncoder()

y_train_adasyn = mapp.fit_transform(y_tr).ravel()

before_counter = Counter(y_tr)
print('Before', before_counter)

ada = ADASYN(random_state=0, sampling_strategy='all')
x_train_ada, y_train_ada = ada.fit_resample(X_tr, y_train_adasyn)

after_counter = Counter(y_train_ada)
print('After', after_counter)

In [ ]:
mapping = dict(zip(mapp.classes_, mapp.transform(mapp.classes_)))
print(mapping)

In [ ]:
x_train_ada = pd.DataFrame(x_train_ada)
y_train_ada = pd.DataFrame(y_train_ada)
y_train_ada.columns = ["Label"]

# Variational autoencoder

In [ ]:
import gc

del encoder, vae
K.clear_session()
gc.collect()

1153

In [ ]:
y_train_ada = pd.read_pickle(f"{path}/pickle/y_train_ada_circle.pkl")
x_train_ada = pd.read_pickle(f"{path}/pickle/x_train_ada_circle.pkl")
x_test = pd.read_pickle(f"{path}/pickle/x_test_circle.pkl")
y_test = pd.read_pickle(f"{path}/pickle/y_test_circle.pkl")
X_val = pd.read_pickle(f"{path}/pickle/X_val_circle.pkl")
y_val = pd.read_pickle(f"{path}/pickle/y_val_circle.pkl")

In [ ]:
# scale data 
from sklearn.preprocessing import MinMaxScaler
t = MinMaxScaler().fit(x_train_ada)
X_train_scaled = t.transform(x_train_ada)
X_val_scaled = t.transform(X_val)
X_test_scaled = t.transform(x_test)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [ ]:
# Function for reparameterization trick to make model differentiable

def sampling(args):
    
    import tensorflow as tf
    # Function with args required for Keras Lambda function
    z_mean, z_log_var = args

    # Draw epsilon of the same shape from a standard normal distribution
    epsilon = K.random_normal(shape=tf.shape(z_mean), mean=0.,
                              stddev=epsilon_std)
    
    # The latent vector is non-deterministic and differentiable
    # in respect to z_mean and z_log_var
    z = z_mean + K.exp(z_log_var / 2) * epsilon
    return z

class CustomVariationalLayer(Layer):
    """
    Define a custom layer that learns and performs the training
    This function is borrowed from:
    https://github.com/fchollet/keras/blob/master/examples/variational_autoencoder.py
    """
    def __init__(self, **kwargs):
        # https://keras.io/layers/writing-your-own-keras-layers/
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return x

class WarmUpCallback(Callback):
    def __init__(self, beta, kappa):
        self.beta = beta
        self.kappa = kappa
    # Behavior on each epoch
    def on_epoch_end(self, epoch, logs={}):
        if K.get_value(self.beta) <= 1:
            K.set_value(self.beta, K.get_value(self.beta) + self.kappa)


In [ ]:
from keras.layers import Input, Dense, Lambda, Activation
from keras.models import Model
from keras import backend as K
from keras import metrics
from tensorflow.python.framework.ops import disable_eager_execution


# define the size of the latent space
original_dim = x_train_ada.shape[1]
latent_dim = 50
batch_size = 512
epochs = 200
learning_rate = 0.00015

epsilon_std = 1.0
beta = K.variable(0)
kappa = 1

In [ ]:
# Encoder
# Input place holder for RNAseq data with specific input size
rnaseq_input = Input(shape=(original_dim, ))

# Input layer is compressed into a mean and log variance vector of size `latent_dim`
# Each layer is initialized with glorot uniform weights and each step (dense connections,
# batch norm, and relu activation) are funneled separately
# Each vector of length `latent_dim` are connected to the rnaseq input tensor
z_mean_dense_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(rnaseq_input)
z_mean_dense_batchnorm = BatchNormalization()(z_mean_dense_linear)
z_mean_encoded = Activation('gelu')(z_mean_dense_batchnorm)

z_log_var_dense_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(rnaseq_input)
z_log_var_dense_batchnorm = BatchNormalization()(z_log_var_dense_linear)
z_log_var_encoded = Activation('gelu')(z_log_var_dense_batchnorm)

# return the encoded and randomly sampled z vector
# Takes two keras layers as input to the custom sampling function layer with a `latent_dim` output
z = Lambda(sampling, output_shape=(latent_dim, ))([z_mean_encoded, z_log_var_encoded])

In [ ]:
# Decoder
decoder_to_reconstruct = Dense(original_dim, kernel_initializer='glorot_uniform', activation='gelu')
rnaseq_reconstruct = decoder_to_reconstruct(z)

In [ ]:
from keras.optimizers import Adam

adam = Adam(lr=learning_rate)
vae_layer = CustomVariationalLayer()([rnaseq_input, rnaseq_reconstruct])
vae = Model(rnaseq_input, vae_layer)
vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

vae.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 4557)]       0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 50)           227900      ['input_2[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 50)           227900      ['input_2[0][0]']                
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 50)          200         ['dense_3[0][0]']                
 rmalization)                                                                               

In [ ]:
disable_eager_execution()

hist = vae.fit(np.array(X_train_scaled),
               epochs=epochs,
               verbose=0,
               batch_size=batch_size,
               validation_data=(np.array(X_val_scaled), None),
               callbacks=[WarmUpCallback(beta, kappa)])

In [ ]:
plot_model(vae)

In [ ]:
plt.plot(hist.history["loss"], label="Training Loss")
plt.plot(hist.history["val_loss"], label="Validation Loss")
plt.legend()

In [ ]:
encoder = Model(rnaseq_input, z_mean_encoded)

In [ ]:
# testing data
x_test_encoded = encoder.predict(X_test_scaled, batch_size=batch_size) # X_test_scaled

temp = pd.DataFrame(x_test_encoded).reset_index()
temp

,index,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,0,0.813978,0.034304,-0.138385,0.023659,0.858496,1.310539,0.085009,0.933837,0.321365,...,0.253905,0.007144,1.337824,0.985740,0.430511,1.903935,2.076302,-0.099974,1.144478,2.720396
1,1,-0.010307,-0.162665,1.371191,0.631584,-0.146819,1.016971,-0.099609,0.092845,-0.073709,...,-0.122062,-0.098201,0.474340,0.198049,-0.036991,0.772392,-0.014499,-0.104503,0.753066,1.086632
2,2,-0.166531,0.883531,-0.149403,-0.021492,-0.124863,0.212593,0.855761,-0.047874,-0.013254,...,0.802752,0.260408,-0.147036,0.195075,1.778080,2.619074,-0.158729,-0.003332,-0.095238,1.131071
3,3,0.315563,0.030576,0.261325,0.492417,2.940787,1.626618,1.686944,0.902159,1.875275,...,-0.164253,0.909141,1.949654,0.387453,0.143623,0.691164,0.989879,0.108470,0.793490,0.227866
4,4,1.445847,-0.094258,-0.043909,0.957756,0.919686,0.997377,1.358455,0.163847,1.129921,...,-0.093112,1.077867,0.326034,0.108830,0.042532,0.324776,1.456211,0.072304,0.500636,0.325651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2151,2151,0.383579,0.014833,0.152904,1.070466,-0.165756,1.781122,1.061129,0.468211,0.652665,...,-0.032927,2.016425,0.876193,0.541602,2.648481,1.429955,0.290847,-0.108674,1.578215,-0.167062
2152,2152,1.605507,0.082936,0.123267,-0.078949,0.368250,-0.029578,-0.169905,0.474656,0.933525,...,-0.151322,0.592680,0.324494,0.861944,-0.166413,1.910821,-0.169140,-0.119289,0.734402,1.532928
2153,2153,0.466124,1.457700,1.812516,2.652836,0.137167,0.892337,0.898731,0.538793,0.041432,...,-0.124383,0.470545,0.376830,0.179856,0.864443,0.392427,0.776329,1.793663,0.539945,0.275950
2154,2154,-0.054760,0.135444,0.203186,0.662257,0.443430,-0.106505,1.768190,0.533879,-0.017878,...,-0.148893,0.123102,1.676513,0.878446,0.191181,-0.001049,0.156658,0.384941,0.582924,0.603407


In [ ]:
silhouetteScore = silhouette_score(temp, y_test["Label"], metric="euclidean")
davies_bouldinScore = davies_bouldin_score(temp, y_test["Label"])

print("Silhuette", silhouetteScore, "davies_bouldinScore", davies_bouldinScore)

Silhuette -0.21981339776353218 davies_bouldinScore 32.232931611864586


In [ ]:
# validation
x_test_encoded = encoder.predict(X_val_scaled, batch_size=batch_size) # X_test_scaled

temp = pd.DataFrame(x_test_encoded).reset_index()

silhouetteScore = silhouette_score(temp, y_val, metric="euclidean")
davies_bouldinScore = davies_bouldin_score(temp, y_val)

print("Silhuette", silhouetteScore, "davies_bouldinScore", davies_bouldinScore)

Silhuette -0.07713830696630734 davies_bouldinScore 363.01564256891925
